In [17]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from io import BytesIO
from IPython.display import display, HTML
import pandas as pd
from sklearn import preprocessing

Train a LSTM to generate music 
music 21 is a module used for dealing with music data
Step1: we use a special music sheet data format which includes instruments, notes and chords called midi
We load the data from a file with a famous japanese composer joe hisaishi's music 
Using the converter.parse function of music 21, we will get a list of all notes and chords in the file.
We append the pitch of the every note object using its string notation
And we append every chord by encoding the id of chords' notes together into one string with each note seperated by a slash.
notes is the processed data. 

In [1]:
notes=[]
offsets=[]
for file in glob.glob("/Users/xiechuxi/Desktop/music/japan/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in flat 
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('|'.join(str(n) for n in element.normalOrder))
    
    
    for element in notes_to_parse:
        offsets.append(element.offset)
     

            


NameError: name 'a' is not defined

Now we get a stream of notes and chords indicated by their notes as characters and chords as number string seperated by '|' (i.e. 1|6|8).
For the following step we will transfer the notes stream of charaters and chords into indices. 

In [112]:
# get all pitch names
pitchnames = sorted(set(item for item in notes))

#map the pitch names into integers
noteindices = dict((note, number) for number, note in enumerate(pitchnames))

#reprocess the whole list into integer indices and seperate the list into input list and outout list we want to predict the next note after each 15 notes 
#but the sequence length could be changed to find the best tune for predict a melody
inputnet = []
outputnet = []
h=[]
sequence_length = 20


for i in range(0, len(notes) - sequence_length, 1):
    sequencein = notes[i:i + sequence_length]
    sequenceout = notes[i + sequence_length]
    for char in sequencein:
        h.append(noteindices[char])
    for j in range(i,i+sequence_length):
                   inputnet.append([h[j],offsets[j]])
    outputnet.append([noteindices[sequenceout],offsets[i+sequence_length]])

#reshape the input and output net into a format compatible with the LSTM net
inputnet = np.reshape(inputnet, (int(len(inputnet)/sequence_length), sequence_length, 2))
#normalize the numerical value into categorical value
inputnet = inputnet
outputnet = np_utils.to_categorical(outputnet)







After the processing of the notes, and seperate the notes into inputnet and outputnet, we can make the prediction. 

In [113]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import numpy as np
from keras.layers import Flatten

# split the input and output nets into test and train set
testinputnet=inputnet[round(4/5*len(inputnet)):len(inputnet)]
testoutputnet=outputnet[round(4/5*len(inputnet)):len(inputnet)]
traininputnet=inputnet[0:round(4/5*len(inputnet))]
trainoutputnet=outputnet[0:round(4/5*len(inputnet))]

print('Build model...')
model = Sequential()
model.add(LSTM(300, dropout=0.0, recurrent_dropout=0.0,input_shape=(traininputnet.shape[1], traininputnet.shape[2])))
model.add(Dense(150, activation='relu'))
model.add(Dense(len(set(notes))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
print('Train...')

model.fit(traininputnet,trainoutputnet,validation_data=(testinputnet,testoutputnet),callbacks=[monitor],verbose=2,epochs=400)

Build model...
Train...


ValueError: Error when checking target: expected activation_2 to have 2 dimensions, but got array with shape (4062, 2, 701)

After the training of the network is done we can start to make a prediction to the music now!(generate music).

In [31]:
from numpy import vstack

seed = np.random.randint(190, size=(20, 1))
#first generate a list of random notes if you want to be more creative with melody 
#you could write a small piece of notes by yourself or copy a piece online.

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = seed#pattern is the iterator which each iteration will drop the first note and predict the next note according to the pattern
predictoutput=[]
#generate 500 notes as seed so that following melody could be created based on this seed
for i in range(400):
    predictinput = np.reshape(pattern, (1, len(pattern), 1))
    predictinput = predictinput 
    prediction = model.predict(predictinput, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    index=np.array([[index]])
    predictoutput.append(result)
    pattern=vstack((pattern, index))
    pattern = pattern[1:len(pattern)]
    


In [28]:
offset = 0
output_notes = []

for pattern in predictoutput:
    # pattern is a chord
    if ('|' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('|')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        #we can actually change the instrument in the further continuation of the project
        output_notes.append(new_note)
    
    offset+=0.5
    #to make sure notes don't stack

In [29]:
from music21 import *
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/Users/xiechuxi/Desktop/music/musiccreate/gn12.mid')

'/Users/xiechuxi/Desktop/music/musiccreate/gn12.mid'